In [1]:
import numpy as np

In [59]:
class Decoder:
    def __init__(self,transition,init_transition,emission,sequence):
        #self.states = states
        self.init_transition = init_transition #normal array
        self.transition = transition #2 dim array, numpy (states x states)
        self.emission = emission #must 2 dim array (states x sequence)
        self.sequence = sequence #observered sequence, normal array

        #Will be used to keep track of everything calculated
        self.matrix = np.zeros((len(emission),len(sequence)))

    def viterbi(self):
        #initiate
        self.matrix[:, 0] = self.init_transition * self.emission[:, self.sequence[0]]
        
        #iteration and keeping the largest argument
        for i in range(1, self.matrix.shape[1]):
            for j in range(self.matrix.shape[0]):
                v_prop = self.matrix[:, i-1] * self.emission[j, i] * self.transition[:, j]
                self.matrix[j, i] = np.max(v_prop)
        #return backp(

In [168]:
class Decoder:
    def __init__(self,transition,init_transition,emission,sequence,states=[]):

        #self.states = states
        self.init_transition = init_transition #normal array
        self.transition = transition #2 dim array, numpy (states x states)
        self.emission = emission #must 2 dim array (states x symbol)
        self.sequence = sequence #observered sequence, normal array
        #This is for cases when states are not given but the number of states is given with the emission array so we utilize that
        if states == []:
            for i in range(emission.shape[0]):
                states.append("state" + str(i))
        #we construct two empty arrays, one for the backpropogation and another to gain the optimum sequence of states
        #could be done in one matrix with dictionaries?
        self.v_matrix = np.zeros((emission.shape[0],len(sequence))) #states x observations
        self.state_index_matrix = np.zeros((emission.shape[1],len(sequence))) #--II--

    def viterbi(self):
        
        #initiate
        self.v_matrix[:, 0] = self.init_transition * self.emission[:, self.sequence[0]]
        #iteration
        for observation in range(1, self.v_matrix.shape[1]): #not the first because that one is initiated
            for state in range(self.v_matrix.shape[0]):
                temp = self.v_matrix[:,observation-1]*self.transition[:,state]
                self.v_matrix[state,observation] = np.argmax(temp)*self.emission[state,self.sequence[observation]]
                #we note the indesxis of what transition where the highest highest value is found
                self.state_index_matrix[state,observation-1] = np.argmax(temp)
        #Backtracking
        back_matrix = np.zeros(len(self.sequence))
        back_matrix[-1] = np.argmax(self.v_matrix[:, -1])
        for n in range(len(self.sequence)-2, -1, -1):
            back_matrix[n] = self.state_index_matrix[int(back_matrix[n+1]), n]
        self.back_matrix = back_matrix
        
                
            
        
        return self.v_matrix, self.state_index_matrix, back_matrix
            
        
        

In [173]:
#taget av chatgpt, ville kolla om allt gick rätt
def generate_hmm_arguments(I, K, N):
    """
    Generate arguments for testing the Viterbi algorithm.

    Args:
        I (int): Number of states.
        K (int): Number of possible observations.
        N (int): Length of the observation sequence.

    Returns:
        A (np.ndarray): State transition probability matrix of dimension I x I.
        C (np.ndarray): Initial state distribution of dimension I.
        B (np.ndarray): Output probability matrix of dimension I x K.
        O (np.ndarray): Observation sequence of length N.
    """
    # Generate random matrices for A, C, and B
    A = np.random.rand(I, I)
    A /= A.sum(axis=1, keepdims=True)  # Normalize rows to ensure valid transition probabilities

    C = np.random.rand(I)
    C /= C.sum()  # Normalize to ensure a valid probability distribution

    B = np.random.rand(I, K)
    B /= B.sum(axis=1, keepdims=True)  # Normalize rows to ensure valid emission probabilities

    # Generate a random observation sequence
    O = np.random.randint(0, K, N)

    return A, C, B, O

# Example usage:
I = 3  # Number of states
K = 4  # Number of possible observations
N = 10  # Length of the observation sequence

A, C, B, O = generate_hmm_arguments(I, K, N)

In [175]:
#just checking if everything is alright

decoder = Decoder(A, C, B, O)

decoder.viterbi()
print("given sequence")
print(O)
print("Resulting Probability path matrix:")
print(decoder.v_matrix)
print("Resulting optimum state sequence matrix")
print(decoder.state_index_matrix)
print("Resulting optimum sequence")
print(decoder.back_matrix)

given sequence
[1 0 1 0 0 1 2 0 1 0]
Resulting Probability path matrix:
[[0.05685424 0.137698   0.52424093 0.137698   0.         0.
  0.         0.         0.         0.        ]
 [0.07692499 0.         0.27990874 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.04969888 0.35286093 0.         0.         0.         0.
  0.         0.         0.         0.        ]]
Resulting optimum state sequence matrix
[[1. 2. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Resulting optimum sequence
[1. 2. 1. 0. 0. 0. 0. 0. 0. 0.]
